In [10]:
# Import in needed libs 
import pandas as pd 
import feedparser

In [11]:
# To get latest data from fuel wacth create a variable to store todays date. 
todays_date = pd.to_datetime("now")
todays_date

Timestamp('2021-11-23 07:55:25.440146')

Bottom Two cells can be run to (cell 1) bulk download and (cell 2) download new updated data.  

In [12]:
# # Run this block to bulk downlaod 
# # create range of monthly dates
# download_dates = pd.date_range(start='01-01-2002',end=todays_date,freq='MS')

# #URL from Chrome DevTools Console
# base_url = ("https://warsydprdstafuelwatch.blob.core.windows.net/historical-reports/FuelWatchRetail-{}.csv") 

# #  create list of remote URL from base URL
# list_of_url = [base_url.format(date.strftime("%m-%Y")) for date in download_dates]
# #print(list_of_url)

# # download and combine multiple files into one DataFrame
# df_bulk = pd.concat((pd.read_csv(url) for url in list_of_url))

In [8]:
# Run this block to download new updated csvs 
# create range of monthly dates
download_dates = pd.date_range(start='01-10-2021',end=todays_date,freq='MS')

#URL from Chrome DevTools Console
base_url = ("https://warsydprdstafuelwatch.blob.core.windows.net/historical-reports/FuelWatchRetail-{}.csv") 

#  create list of remote URL from base URL
list_of_url = [base_url.format(date.strftime("%m-%Y")) for date in download_dates]
#print(list_of_url)

# download and combine multiple files into one DataFrame
df_bulk = pd.concat((pd.read_csv(url) for url in list_of_url))

df_bulk.head()

,PUBLISH_DATE,TRADING_NAME,BRAND_DESCRIPTION,PRODUCT_DESCRIPTION,PRODUCT_PRICE,ADDRESS,LOCATION,POSTCODE,AREA_DESCRIPTION,REGION_DESCRIPTION,Unnamed: 10
0,01/02/2021,53 Mile Roadhouse,United,ULP,116.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,NaN
1,01/02/2021,53 Mile Roadhouse,United,Diesel,115.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,NaN
2,01/02/2021,53 Mile Roadhouse,United,98 RON,131.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,NaN
3,01/02/2021,7-Eleven Ascot,7-Eleven,ULP,112.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro,NaN
4,01/02/2021,7-Eleven Ascot,7-Eleven,Brand Diesel,123.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro,NaN


In [9]:
# Noticed a extra 'Unnamed: 10' column that had no values so dropped it. 
perth_fuel = df_bulk.drop(columns='Unnamed: 10')

•	Noticed all headers where in capital letters for easier references dropped them down to lower case. 

•	Also for easier references (Eg searching by year, month, day) decided to split the ‘publish date’ column into year , month and day. 

•	Dropped the ‘publish date’ it’s not required after the split 

In [13]:
# Drop the headers to lower case. 
perth_fuel.columns = [str(x).lower() for x in perth_fuel.columns]

# Split the 'publish_date' into 3 columns. 
perth_fuel[['day', 'month', 'year']] = perth_fuel['publish_date'].str.split('/', expand = True)

# Drop the 'publish_date' column.
perth_fuel = perth_fuel.drop(columns='publish_date')

perth_fuel.head()

,trading_name,brand_description,product_description,product_price,address,location,postcode,area_description,region_description,day,month,year
0,53 Mile Roadhouse,United,ULP,116.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,01,02,2021
1,53 Mile Roadhouse,United,Diesel,115.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,01,02,2021
2,53 Mile Roadhouse,United,98 RON,131.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel,01,02,2021
3,7-Eleven Ascot,7-Eleven,ULP,112.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro,01,02,2021
4,7-Eleven Ascot,7-Eleven,Brand Diesel,123.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro,01,02,2021


•	Noticed that the new date columns are located at the end of the data frame move them to front. 

In [14]:
# Re-order dataframe for logic bases move 'year', 'month' and 'day' to the left. 
perth_fuel = perth_fuel[[
    'year',
    'month',
    'day',
    'trading_name', 
    'brand_description',
    'product_description',
    'product_price',
    'address',
    'location',
    'postcode',
    'area_description',
    'region_description']]

perth_fuel.head()

,year,month,day,trading_name,brand_description,product_description,product_price,address,location,postcode,area_description,region_description
0,2021,02,01,53 Mile Roadhouse,United,ULP,116.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel
1,2021,02,01,53 Mile Roadhouse,United,Diesel,115.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel
2,2021,02,01,53 Mile Roadhouse,United,98 RON,131.5,31 South Western Hwy,PINJARRA,6208,Murray,Peel
3,2021,02,01,7-Eleven Ascot,7-Eleven,ULP,112.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro
4,2021,02,01,7-Eleven Ascot,7-Eleven,Brand Diesel,123.9,194 Great Eastern Hwy,ASCOT,6104,South of River,Metro


In [15]:
perth_fuel.dtypes

year                    object
month                   object
day                     object
trading_name            object
brand_description       object
product_description     object
product_price          float64
address                 object
location                object
postcode                 int64
area_description        object
region_description      object
dtype: object

In [16]:
# Change the dtype to numeric 
perth_fuel[["year", "month", "day"]] = perth_fuel[["year", "month", "day"]].apply(pd.to_numeric)

In [17]:
#Get a list of number of brands in WA. 
number_brands = pd.DataFrame(perth_fuel['brand_description'].unique(), columns=['brand'])
print(number_brands.count())

brand    25
dtype: int64


In [18]:
# Get a list of number of stations in WA. 
number_station = pd.DataFrame(perth_fuel['trading_name'].unique(),columns=['trading_name'])
#number_station.rename(columns={'0':'Brands'}, inplace=True)
print(number_station.count())

trading_name    804
dtype: int64


In [19]:
# List of Regions in WA. 
number_regions = pd.DataFrame(perth_fuel['region_description'].unique(), columns=['region'])
print(number_regions.count())

region    10
dtype: int64


In [20]:
# list the number of suburbs in WA. 
number_suburbs = pd.DataFrame(perth_fuel['area_description'].unique(), columns=['suburb'])
print(number_suburbs.count())

suburb    57
dtype: int64


•	Moving the data onto the AWS database so it can be used through Tableau 

In [21]:
perth_fuel = perth_fuel.reset_index()
perth_fuel = perth_fuel.drop(columns=['index'])
# perth_fuel.to_csv('perthfuel.csv')

•	Due to the size of the database and how long it was taking to pull the data best to break down into different tables to build charts

In [22]:
# Create a dataframe that stores the values by year using ‘groupby’
year = perth_fuel.groupby(["year","region_description","area_description","brand_description","product_description"])["product_price"].mean()
year = pd.DataFrame(year)
year = year.reset_index()

# Round the values to 3. 
year = year.round({'product_price':3})
#year.to_csv('year.csv')
year.head()

,year,region_description,area_description,brand_description,product_description,product_price
0,2021,Gascoyne,Carnarvon,Ampol,Brand Diesel,175.419
1,2021,Gascoyne,Carnarvon,Ampol,Diesel,172.419
2,2021,Gascoyne,Carnarvon,BP,98 RON,172.117
3,2021,Gascoyne,Carnarvon,BP,Diesel,150.408
4,2021,Gascoyne,Carnarvon,BP,ULP,152.117


In [23]:
# Create a dataframe that stores the values by year/month using ‘groupby’
month = perth_fuel.groupby(["year","month","region_description","area_description","brand_description","product_description"])["product_price"].mean()
month = pd.DataFrame(month)
month = month.reset_index()

# Round the values to 3 
month = month.round({'product_price':3})
#month.to_csv('month.csv')
month.head()

,year,month,region_description,area_description,brand_description,product_description,product_price
0,2021,2,Gascoyne,Carnarvon,BP,98 RON,150.793
1,2021,2,Gascoyne,Carnarvon,BP,Diesel,134.043
2,2021,2,Gascoyne,Carnarvon,BP,ULP,130.793
3,2021,2,Gascoyne,Carnarvon,Caltex,98 RON,153.757
4,2021,2,Gascoyne,Carnarvon,Caltex,Brand Diesel,137.596


•	This section was created to try and get live data onto the website using the available RSS feed. 

In [35]:
# The rss feed link from the fuelwatch website. 
rawrss = ['https://www.fuelwatch.wa.gov.au/fuelwatch/fuelWatchRSS?StateRegion=98&Product=1&Day=tomorrow']

# Create datframe to sotre the data from the rss feed. 
df_rss = pd.DataFrame([])

# Create a for loop to grab the diffrent types of information. 
for url in rawrss:
    dp = feedparser.parse(url)

    for i, e in enumerate(dp.entries):
        feed = {}
        feed['Price'] = e.price if 'price' in e else f'price {i}'
        feed['Brand'] = e.brand if 'brand' in e else f'price {1}'
        feed['Address'] = e.address if 'address' in e else f'no address {i}'
        feed['Phone'] = e.phone if 'phone' in e else f'phone {i}'
        df = df.append(pd.DataFrame([feed]), ignore_index=True)

df_rss.head()

""


In [37]:
# Check the data types. 
df_rss.head()

""


In [30]:
# Change the numeric values again 
df_rss['price'] = pd.to_numeric(df_rss['price'])

KeyError: 'price'

In [31]:
# Get the average and greate dataframe to store it also keep three values, 
data_mean = pd.DataFrame(df_rss.mean(), columns=['Average ULP Perth Metro'])
df_rss = df_rss.nsmallest(3, 'Price')

KeyError: 'Price'

In [ ]:
rawrss = ['https://www.fuelwatch.wa.gov.au/fuelwatch/fuelWatchRSS?StateRegion=98&Product=1&Day=tomorrow']

best_metro_ulp_price = pd.DataFrame([])

for url in rawrss:
    dp = feedparser.parse(url)

    for i, e in enumerate(dp.entries):
        one_feed = {}
        one_feed['Price'] = e.price if 'price' in e else f'price {i}'
        one_feed['Brand'] = e.brand if 'brand' in e else f'price {1}'
        one_feed['Address'] = e.address if 'address' in e else f'no address {i}'
        one_feed['Phone'] = e.phone if 'phone' in e else f'phone {i}'
        best_metro_ulp_price = best_metro_ulp_price.append(pd.DataFrame([one_feed]), ignore_index=True)

best_metro_ulp_price.head()

In [ ]:
best_metro_ulp_price['Price'] =pd.to_numeric(best_metro_ulp_price['Price'])
top_1 = best_metro_ulp_price.nsmallest(1,'Price').drop(columns=['Address', 'Phone'])